In [1]:
# ============================================================
# GRU + Off-Season Cross-Attention (Opponent Scouting)
# ============================================================

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, MultiHeadAttention, LayerNormalization, Dropout
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [17]:
# ---------------------------
# Load Data
# ---------------------------
gru_df = pd.read_csv("gru_team_embeddings_by_week.csv")
games_df = pd.read_excel("nfl_2024_schedule.xlsx")



gru_cols = [c for c in gru_df.columns if c.startswith("emb_")]
gru_dim = len(gru_cols)

games_df = games_df.rename(columns={"Home Team": "home", "Away Team": "away", "Seasonweek": "week"})
games_df["week"] = games_df["week"].astype(int)

# ✅ Only keep games from Week 202318 onward
start_week = 202318
games_df = games_df[games_df["week"] >= start_week].copy()

print(f"✅ Loaded GRU embeddings ({len(gru_df)} rows, {gru_dim} features)")
print(f"✅ Loaded {len(games_df)} games from week {start_week} onward")

✅ Loaded GRU embeddings (1598 rows, 20 features)
✅ Loaded 272 games from week 202318 onward


In [18]:
# ------------------------------------------------------------
# 2. Load and Scale Off-Season Features
# ------------------------------------------------------------
off_df = pd.read_csv("offseason_2024_labeled.csv")
off_df['Team'] = ['CRD', 'ATL', 'RAV', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
              'DEN', 'DET', 'GNB', 'HTX', 'CLT', 'JAX', 'KAN', 'RAI', 'SDG',
              'RAM', 'MIA', 'MIN', 'NWE', 'NOR', 'NYG', 'NYJ', 'PHI', 'PIT',
              'SFO', 'SEA', 'TAM', 'OTI', 'WAS'] 

# Drop label and team columns if present
off_df = off_df.drop(columns=["Improvement?"], errors="ignore")

team_col = "Team" if "Team" in off_df.columns else "team"
off_df[team_col] = off_df[team_col].str.strip()

# Identify which columns to scale
cat_features = ["New Coach", "New QB"]
num_features = [c for c in off_df.columns if c not in cat_features + [team_col]]

scaler = StandardScaler()
off_df_scaled = off_df.copy()
off_df_scaled[num_features] = scaler.fit_transform(off_df[num_features])

off_cols = num_features + cat_features
print(f"✅ Off-season data scaled ({len(off_cols)} features, {len(cat_features)} categorical untouched)")

✅ Off-season data scaled (12 features, 2 categorical untouched)


In [19]:
# ------------------------------------------------------------
# 3. Define Mutual Cross-Attention Layer
# ------------------------------------------------------------
class ScoutingAttention(Layer):
    """Mutual cross-attention where Away and Home teams contextualize each other."""
    def __init__(self, num_heads=4, key_dim=8, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout)
        self.norm_q = LayerNormalization()
        self.norm_kv = LayerNormalization()
        self.dropout = Dropout(dropout)

    def call(self, away_team, home_team):
        attn_away = self.attn(
            query=self.norm_q(away_team),
            value=self.norm_kv(home_team),
            key=self.norm_kv(home_team)
        )
        attn_home = self.attn(
            query=self.norm_q(home_team),
            value=self.norm_kv(away_team),
            key=self.norm_kv(away_team)
        )
        return tf.concat([self.dropout(attn_away), self.dropout(attn_home)], axis=-1)

scouting_layer = ScoutingAttention(num_heads=4, key_dim=8, dropout=0.1)

In [24]:
import warnings
warnings.filterwarnings('ignore')
# ------------------------------------------------------------
# 4. Build Fused Embeddings (GRU + Off-Season)
# ------------------------------------------------------------
results = []
for _, row in tqdm(games_df.iterrows(), total=len(games_df), desc=f"Generating Cross-Attention from {start_week}"):
    week = int(row["week"])
    away, home = row["away"], row["home"]

    # --- Retrieve GRU embeddings ---
    away_gru = gru_df[(gru_df["team"] == away) & (gru_df["seasonweek"] == week)][gru_cols].values
    home_gru = gru_df[(gru_df["team"] == home) & (gru_df["seasonweek"] == week)][gru_cols].values
    if away_gru.size == 0 or home_gru.size == 0:
        continue

    # --- Retrieve off-season features ---
    away_off = off_df_scaled[off_df_scaled[team_col] == away][off_cols].values
    home_off = off_df_scaled[off_df_scaled[team_col] == home][off_cols].values
    if away_off.size == 0 or home_off.size == 0:
        continue

    # --- Concatenate GRU + Off-Season ---
    away_vec = np.concatenate([away_gru.flatten(), away_off.flatten()])[None, :]
    home_vec = np.concatenate([home_gru.flatten(), home_off.flatten()])[None, :]

    # --- Tensor format for attention ---
    away_tf = tf.expand_dims(tf.convert_to_tensor(away_vec, dtype=tf.float32), axis=1)
    home_tf = tf.expand_dims(tf.convert_to_tensor(home_vec, dtype=tf.float32), axis=1)

    # --- Mutual attention ---
    attended = scouting_layer(away_tf, home_tf)
    attended_np = tf.squeeze(attended).numpy()

    # --- Split back into away/home halves ---
    half = attended_np.shape[0] // 2
    away_half, home_half = attended_np[:half], attended_np[half:]

    winner = None
    if "Winner" in row:
        winner = row["Winner"]
    elif "winner" in row:
        winner = row["winner"]

    results.append({
        "week": week,
        "away_team": away,
        "home_team": home,
        "winner": winner,
        **{f"A_attn_{i}": away_half[i] for i in range(len(away_half))},
        **{f"B_attn_{i}": home_half[i] for i in range(len(home_half))}
    })

Generating Cross-Attention from 202318: 100%|████████████████████████████████████████| 272/272 [00:06<00:00, 43.29it/s]


In [25]:
# ------------------------------------------------------------
# 5. Save to CSV
# ------------------------------------------------------------
crossattn_df = pd.DataFrame(results).sort_values("week").reset_index(drop=True)
crossattn_df.to_csv("crossattn_gru_offseason_embeddings.csv", index=False)

print(f"📁 Saved {len(crossattn_df)} rows → crossattn_gru_offseason_embeddings.csv")
print("Columns:", crossattn_df.columns.tolist()[:10], "...")

📁 Saved 272 rows → crossattn_gru_offseason_embeddings.csv
Columns: ['week', 'away_team', 'home_team', 'winner', 'A_attn_0', 'A_attn_1', 'A_attn_2', 'A_attn_3', 'A_attn_4', 'A_attn_5'] ...


In [26]:
crossattn_df.head()

,week,away_team,home_team,winner,A_attn_0,A_attn_1,A_attn_2,A_attn_3,A_attn_4,A_attn_5,...,B_attn_22,B_attn_23,B_attn_24,B_attn_25,B_attn_26,B_attn_27,B_attn_28,B_attn_29,B_attn_30,B_attn_31
0,202401,WAS,TAM,TAM,0.157442,0.118900,0.020763,-0.146249,0.283556,0.138088,...,0.302581,0.451895,-0.557338,-0.264546,-0.393765,-0.047967,0.389524,-0.157454,-0.335450,0.029156
1,202401,RAM,DET,DET,-0.219006,0.092546,0.174104,-0.137766,-0.090072,-0.312069,...,-0.290184,0.251080,0.017910,0.129673,-0.224438,0.378855,-0.133719,0.524660,-0.302816,0.705316
2,202401,NYJ,SFO,SFO,0.347006,0.086024,0.107557,0.282922,0.230930,0.180891,...,0.299523,0.176346,-0.195049,-0.376100,-0.165773,-0.241895,0.144016,-0.252767,0.394055,0.061410
3,202401,OTI,CHI,CHI,0.119397,-0.055279,0.030072,0.456825,0.002875,-0.362278,...,-0.145163,-0.082446,-0.129973,0.293424,-0.123833,0.010067,0.643185,0.132929,-0.561679,0.336049
4,202401,JAX,MIA,MIA,-0.022381,-0.103671,0.189966,0.306426,-0.238073,0.097357,...,0.223243,-0.066203,0.580412,0.065606,0.246170,-0.156800,-0.197900,-0.207810,0.427105,0.403887
